# Ngram language model lab

In this lab you will do 4 exercises building and evaluating ngram language models of the following types:
1. A Maximum Liklihood Estimation (MLE) unigram model (10 marks)
2. A bigram model with add-one Laplace smoothing (10 marks)
3. A bigram model with general additive (add-k) smoothing (10 marks)
4. Ngram models with an advanced interpolation technique, Kneser-Ney snoothing (the methods are provided) (10 marks)

Before you start the exercises, make sure you run and understand the examples first. Then complete the exercises using the following 3 files with line-separated text to train the bigger language models on:
* training data -- "switchboard_lm_training.txt"
* heldout/development data -- "switchboard_lm_heldout.txt"
* test data -- "switchboard_lm_test.txt"

In [1]:
from __future__ import division  # for python 2 this is needed
from __future__ import print_function # for python 2 this is needed
from collections import Counter
from math import log

In [2]:
# Some useful methods for all exercises:
def glue_tokens(tokens, order):
    """A useful way of glueing tokens together for
    Kneser Ney smoothing and other smoothing methods
    
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    return '{0}@{1}'.format(order,' '.join(tokens))

def unglue_tokens(tokenstring, order):
    """Ungluing tokens glued by the glue_tokens method"""
    if order == 1:
        return [tokenstring.split("@")[1].replace(" ","")]
    return tokenstring.split("@")[1].split(" ")

def tokenize_sentence(sentence, order):
    """Returns a list of tokens with the correct numbers of initial
    and end tags (this is meant ot be used with a non-backoff model!!!)
    
    :sentence: a string of text
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    tokens = sentence.split()
    tokens = ['<s>'] * (order-1) + tokens + ['</s>']
    return tokens

In [37]:
sentence = "lets order beer"
order = 3
tokenize_sentence(sentence, order)

['<s>', '<s>', 'lets', 'order', 'milkshake', '</s>']

# Examples

In [3]:
# Set of sentences (corpus) from the example in the lecture slides
sentences = [
            "I am Sam",
            "Sam I am",
            "I do not like green eggs and ham"
            ]

## Example 1. Build a unigram MLE language model from a simple corpus.
An MLE unigram model will tell you how likely a word is to occur, estimated from the function of counts:

C(w_i)/N

where C(w_i) is the number of times the word at position i occurred in the training corpus, and N is the sum of the counts of all words, or, to put it another way, the length of the training corpus.

Notice the tokenization method adds a `</s>` at the end but no `<s>` is needed at the beginning of each sentence
    because unigrams do not have a context word (we are only concerned with the frequency of single words).

In [4]:
unigrams = Counter()
for sent in sentences:
    words = tokenize_sentence(sent, 1)
    print("tokenized", words)
    for w in words:
        unigrams[w] +=1
unigram_total = sum(unigrams.values()) # to get the denominator for unigram probabilities
print (len(unigrams), "different unigrams observed")
print("unigram total", unigram_total)

tokenized ['I', 'am', 'Sam', '</s>']
tokenized ['Sam', 'I', 'am', '</s>']
tokenized ['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>']
11 different unigrams observed
unigram total 17


To evaluate the model, we will measure the model's perplexity of those same training sentences. Note in normal practice you would want to do this on different data (as you will do below).

Perplexity is equal to 2 to the power of the cross entropy where cross entropy is the negative sum of all log probabilities from the model normalized by the length of the corpus N.

Measure the cross entropy and perplexity on each sentence too.

In [5]:
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s 
for sent in sentences:
    # get the unigram model based probability of each sentence
    words = tokenize_sentence(sent, 1) # tokenize sentence with the order 1 as the parameter
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for w in words:
        prob = unigrams[w]/unigram_total
        logprob = log(prob, 2)  # the log of the prob to base 2
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("unigram corpus cross entropy", cross_entropy)
print("unigram corpus perplexity", perplexity)

['I', 'am', 'Sam', '</s>'] cross entropy: 2.7949815908897615 perplexity: 6.940220937885672
['Sam', 'I', 'am', '</s>'] cross entropy: 2.7949815908897615 perplexity: 6.940220937885672
['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] cross entropy: 3.7352489522011934 perplexity: 13.317477627933627
unigram corpus cross entropy 3.2927701939369913
unigram corpus perplexity 9.799921507045037


## Example 2. Build a bigram MLE language model from the same corpus
A MLE (unsmoothed) bigram model will tell you how likely a word is to occur given the previous word, estimated from the function of counts:

C(w_i-1, w_i)/C(w_i)

where C(w_i-1, w_i) is the number of times the word at position i follows the word at position i-1 in the training corpus, and N is the sum of the counts of all words (or, to put it another way, the length of the training corpus).

Notice the tokenization method adds a `</s>` at the end and also one `<s>` for padding at the beginning as we want to count the number of times the word at the beginning of each sentence begins a sentence.

In [6]:
# First get the counts from the training corpus for bigrams without smoothing
bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
delta = 1  # delta is order - 1
for s in sentences:
    words = tokenize_sentence(s, 2)  # tokenize sentence with the order 2 as the parameter
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        bigrams[glue_tokens(ngram, 2)] +=1
        bigram_context[glue_tokens(context, 1)] += 1
print(len(bigrams.keys()), "different bigrams")
print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

15 different bigrams
11 different bigram contexts (and unigrams) observed


In [7]:
# a handy function to calculate the probability of an bigram from the counts
def prob_bigram_MLE(ngram):
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = numerator / denominator
    return prob


# check if each bigram continuation distribution sums to one
# look at the distributions of possible continuations after each word
for context, v in bigram_context.items():
    context = unglue_tokens(context, 1)
    print("%% context", context)
    check_ngram_total_sums_to_1 = 0
    # for a given context the continuation probabilities 
    # over the whole vocab should sum to 1
    for u in unigrams.keys():
        ngram = context + [u]
        prob = prob_bigram_MLE(ngram)
        print(ngram, prob)
        check_ngram_total_sums_to_1 += prob
    print("sums to 1?", check_ngram_total_sums_to_1)

%% context ['<s>']
['<s>', 'I'] 0.6666666666666666
['<s>', 'am'] 0.0
['<s>', 'Sam'] 0.3333333333333333
['<s>', '</s>'] 0.0
['<s>', 'do'] 0.0
['<s>', 'not'] 0.0
['<s>', 'like'] 0.0
['<s>', 'green'] 0.0
['<s>', 'eggs'] 0.0
['<s>', 'and'] 0.0
['<s>', 'ham'] 0.0
sums to 1? 1.0
%% context ['I']
['I', 'I'] 0.0
['I', 'am'] 0.6666666666666666
['I', 'Sam'] 0.0
['I', '</s>'] 0.0
['I', 'do'] 0.3333333333333333
['I', 'not'] 0.0
['I', 'like'] 0.0
['I', 'green'] 0.0
['I', 'eggs'] 0.0
['I', 'and'] 0.0
['I', 'ham'] 0.0
sums to 1? 1.0
%% context ['am']
['am', 'I'] 0.0
['am', 'am'] 0.0
['am', 'Sam'] 0.5
['am', '</s>'] 0.5
['am', 'do'] 0.0
['am', 'not'] 0.0
['am', 'like'] 0.0
['am', 'green'] 0.0
['am', 'eggs'] 0.0
['am', 'and'] 0.0
['am', 'ham'] 0.0
sums to 1? 1.0
%% context ['Sam']
['Sam', 'I'] 0.5
['Sam', 'am'] 0.0
['Sam', 'Sam'] 0.0
['Sam', '</s>'] 0.5
['Sam', 'do'] 0.0
['Sam', 'not'] 0.0
['Sam', 'like'] 0.0
['Sam', 'green'] 0.0
['Sam', 'eggs'] 0.0
['Sam', 'and'] 0.0
['Sam', 'ham'] 0.0
sums to 1? 1.0


In [8]:
# Check the estimates for the lecture examples:
# p(I|<s>)
# p(Sam|<s>)
# p(am|I)
# p(</s>|Sam)
# p(Sam|am)
# p(do|I)

print(prob_bigram_MLE(['<s>','I']))
print(prob_bigram_MLE(['<s>', 'Sam']))
print(prob_bigram_MLE(['I', 'am']))
print(prob_bigram_MLE(['Sam', '</s>']))
print(prob_bigram_MLE(['am', 'Sam']))
print(prob_bigram_MLE(['I', 'do']))

0.6666666666666666
0.3333333333333333
0.6666666666666666
0.5
0.5
0.3333333333333333


To evaluate the model, as in the unigram case above we will measure the model's perplexity of those same training sentences.

Notice that even with this small corpus the bigram perplexity is significantly lower than the unigram perplexity.

In [9]:

s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s
for sent in sentences:
    words = tokenize_sentence(sent, 2)
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        prob = prob_bigram_MLE(ngram)
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("bigram corpus cross entropy", cross_entropy)
print("bigram corpus perplexity", perplexity)

['<s>', 'I', 'am', 'Sam', '</s>'] cross entropy: 0.7924812503605781 perplexity: 1.7320508075688774
['<s>', 'Sam', 'I', 'am', '</s>'] cross entropy: 1.042481250360578 perplexity: 2.0597671439071177
['<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] cross entropy: 0.24110277793803472 perplexity: 1.1818957424692271
bigram corpus cross entropy 0.5593985296662904
bigram corpus perplexity 1.4736547115524326


# Exercises

# Exercise 1. Unigram MLE model from a bigger corpus

Write code to read in the file `switchboard_language_model_train.txt` which has preprocessed text on each line.

You will populate a unigram language model based on that data for an MLE estimation using a `Counter` using this data (see Example 1 above as to how this is done for a smaller dataset).

Before you do this, you have to define a vocabulary of words using the training data, which you will keep the same for all of the following exercises. In these exercises, the vocabulary is defined by using a Minimum Document Frequency of 2 in the training data. That means the vocab should only contain words which occur at least twice in the training data.

Any words not in the vocabulary in the training, heldout and testing data must be replaced with an out-of-vocab symbol `<unk/>` before processing them.
 
Using this model, calculate the perplexity of the ENTIRE test corpus `switchboard_language_model_test.txt`- again, remember to replace words unknown by the model with `<unk/>` before getting these measures. See Example 1 as to how this is done for a unigram model on the smaller example dataset.

1. Read text from `switchboard_language_model_train.txt`
2. Define vocab of words using training data, where freq of each word has to be greater than 2.
3. Build unigram language model
4. calculate perplexity on `switchboard_language_model_train.txt`and replace unknown words with `<unk/>`.   

In [10]:
# read the training file in a list `switchboard_lm_train_list` and clean \n 
switchboard_lm_train_list = [i.replace('\n','') for i in open('switchboard_lm_train.txt','r').readlines()]
print('Total Number of sentences in Training: ',len(switchboard_lm_train_list))
print('First 10 sentences: ',switchboard_lm_train_list[0:10])


Total Number of sentences in Training:  175335
First 10 sentences:  ['whos your favorite team', 'my favorite team is the $unc$pittsburgh $unc$steelers', '$unc$pittsburgh', 'you bet', 'i used to be a big $unc$pittsburgh fan when i was little', 'i', 'when $unc$john $unc$stallworth played he was from my hometown in $unc$alabama', 'really', 'so i kind of grabbed on to that team way back when', 'thats pretty nice']


In [11]:
switchboard_lm_heldout_list = [i.replace('\n','') for i in open('switchboard_lm_heldout.txt','r').readlines()]
print('Total Number of sentences in heldout: ',len(switchboard_lm_heldout_list))
print('First 10 sentences: ',switchboard_lm_heldout_list[0:10])

Total Number of sentences in heldout:  5561
First 10 sentences:  ['do you know anyone that is in a nursing home or has ever been in $unc$one', 'no', 'but my grandparents were looking into it before', 'so i know what theyve said', 'uh-huh', 'uh-huh', 'im trying to think', 'my wifes grandmother had alzheimers', 'and they were going to put her into a nursing home', 'and when they put her in she had all kinds of trouble']


In [12]:
switchboard_lm_test_list = [i.replace('\n','') for i in open('switchboard_lm_test.txt','r').readlines()]
print('Total Number of sentences in Test: ',len(switchboard_lm_test_list))
print('First 10 sentences: ',switchboard_lm_test_list[0:10])

Total Number of sentences in Test:  5744
First 10 sentences:  ['the question was kind of interesting to me because i was just trying to put together a long term financial plan and monthly budget', 'the only thing i do now is put the data into $unc$quicken', 'i dont know if you are familiar with that', 'yeah', 'i have some friends of mine who use $unc$quicken', 'and ive considered using it once myself', 'but i decided that the amount of information that would have to go in would be a lot of time keeping that up to date', 'uh-huh', 'so i kind of gave up on the idea of using $unc$quicken at least for now', 'ive found its the only reliable way to keep a check book balanced actually because what will happen is my wife will write a few checks and then not bother to total it']


In [13]:
# generate vocabulary in vocab_words list where frequency of each word is greater than or equal to 2
vocab_words = []
vocab = {}
count = 0
single_word_counter = Counter()
for sent in switchboard_lm_train_list:
    words = sent.split()
    count = count +1
    vocab['<s>'] = count
    for w in words:
        single_word_counter[w] +=1

# print(single_word_counter)
count = 0

for k,v in single_word_counter.items():
  if v >= 2:
    vocab[k] = v
  else:
    count = count + 1
    vocab['<unk/>'] = count 
  
vocab_words = list(vocab.keys())
print(vocab_words[0:10])
print(len(vocab_words))
# vocab['<s/>']

['<s>', 'whos', 'your', 'favorite', 'team', 'my', 'is', 'the', '$unc$pittsburgh', '$unc$steelers']
12730


In [14]:
def in_vocab(word_list):
  """
  Returns a sentence with words which are in vocab otherwise replace the word with <unk/>.

  :word_list: a sequence of words forming a sentence
  """
  ret_word_list = []
  for i in word_list:
    if i not in vocab_words:
      i = '<unk/>'
    ret_word_list.append(i)
  return ' '.join(ret_word_list)

In [15]:
switchboard_lm_train_oov = []
for sent in switchboard_lm_train_list:
  words = sent.split()
  switchboard_lm_train_oov.append(in_vocab(words)) 

print(switchboard_lm_train_oov[40:60])  

['my <unk/>', 'those big piece of something came falling out of the roof and landed on the field', 'so theres this big fire on the field', 'and they were dumping the <unk/> bucket and everything', 'i cant believe that', 'it was <unk/>', 'you just really cant tell whats going to happen', 'thats hilarious', 'i know', 'its like about $unc$two weeks ago i was watching the $unc$saints', 'uh-huh', 'and $unc$morton $unc$anderson kicked a $unc$sixty yard field goal', 'oh', 'and it was beautiful right down the middle', 'boy', 'he is tough', 'he has an incredible leg', 'he is', 'yes', 'do you think theyre going to be able to make it this year past the first playoff game']


In [16]:
switchboard_lm_test_oov = []
for sent in switchboard_lm_test_list:
  words = sent.split()
  switchboard_lm_test_oov.append(in_vocab(words))   

print(switchboard_lm_test_oov[50:60]) 

['<unk/>', 'okay', 'how about you', '$unc$cincinnati actually', 'i was kind of wondering if they would be collecting people with the western accents or something like that', 'i dont know how many people are getting involved in this', 'but thats interesting', 'uh-huh', 'i was kind of interested if you found out about the study by reading $unc$telecom $unc$digest', 'or was there another']


In [17]:
switchboard_lm_heldout_oov = []
for sent in switchboard_lm_heldout_list:
  words = sent.split()
  switchboard_lm_heldout_oov.append(in_vocab(words))   

print(switchboard_lm_heldout_oov[10:20]) 

['and the nursing home made them come and take her back because she was being a <unk/> or worse than a <unk/>', 'she sort of went bananas', 'they thought it was too much of a bother', 'and they couldnt deal with her', 'so i guess you need to know whether its a no deposit no return kind of thing whether', 'right', 'if theyre equipped to handle the kind of patients youre going to have put in there too', 'yeah', 'because i know my grandparents wanted to have some independence still', 'right']


In [18]:
unigrams = Counter()
for sent in switchboard_lm_train_oov:
    words = tokenize_sentence(sent, 1)
    #print("tokenized", words)
    for w in words:
        unigrams[w] +=1
unigram_total = sum(unigrams.values()) # to get the denominator for unigram probabilities
print (len(unigrams), "different unigrams observed")
print("unigram total", unigram_total)

12730 different unigrams observed
unigram total 1306313


In [19]:
#vocab['<s/>']

In [20]:
sentences = switchboard_lm_test_oov
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s 
for sent in sentences:
    # get the unigram model based probability of each sentence
    words = tokenize_sentence(sent, 1) # tokenize sentence with the order 1 as the parameter
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for w in words:
        prob = unigrams[w]/unigram_total
        logprob = log(prob, 2)  # the log of the prob to base 2
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    #print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("unigram corpus cross entropy", cross_entropy)
print("unigram corpus perplexity", perplexity)

unigram corpus cross entropy 8.287702715147693
unigram corpus perplexity 312.4979066155026


including `<unk/>` in vocab

12730 different unigrams observed

unigram total 1306313

unigram corpus cross entropy 8.28770271515

unigram corpus perplexity 312.497906616


# Exercise 2. Bigram model with add-one smoothing

Change your method for reading in and training a language model from Exercise 1 so it works for bigrams. Use the same methods for identifying and replacing out-of-vocabulary words as you did in Exercise 1.

However, in testing, using these raw counts rather than simply implementing MLE you should implement add-one smoothing (see the lecture notes and Jurafsky & Martin Chapter 3/Manning and Schuetze Chapter 6). Remember this involves using the vocabulary size.

Obtain the perplexity score on the test data as above for this smoothed bigram model.

In [21]:
sentences = switchboard_lm_train_oov
# First get the counts from the training corpus for bigrams without smoothing
bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
delta = 1  # delta is order - 1
for s in sentences:
    words = tokenize_sentence(s, 2)  # tokenize sentence with the order 2 as the parameter
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        bigrams[glue_tokens(ngram, 2)] +=1
        bigram_context[glue_tokens(context, 1)] += 1
print(len(bigrams.keys()), "different bigrams")
print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

211560 different bigrams
12730 different bigram contexts (and unigrams) observed


In [22]:
len(vocab_words)

12730

In [23]:
# a handy function to calculate the probability of an bigram from the counts
def prob_bigram_MLE(ngram):
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)] + 1
    denominator = bigram_context[glue_tokens(ngram[:1], 1)] + len(vocab_words)
    prob = numerator / denominator
    return prob


# # check if each bigram continuation distribution sums to one
# # look at the distributions of possible continuations after each word
# for context, v in bigram_context.items():
#     context = unglue_tokens(context, 1)
#     #print("%% context", context)
#     check_ngram_total_sums_to_1 = 0
#     # for a given context the continuation probabilities 
#     # over the whole vocab should sum to 1
#     for u in unigrams.keys():
#         ngram = context + [u]
#         prob = prob_bigram_MLE(ngram)
#         #print(ngram, prob)
#         check_ngram_total_sums_to_1 += prob
#     #print("sums to 1?", check_ngram_total_sums_to_1)

In [24]:
sentences = switchboard_lm_test_oov
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s
for sent in sentences:
    words = tokenize_sentence(sent, 2)
    #print(words)
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        #print(ngram)
        prob = prob_bigram_MLE(ngram)
        #print(prob)
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    #print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("bigram corpus cross entropy", cross_entropy)
print("bigram corpus perplexity", perplexity)

bigram corpus cross entropy 8.248721413034907
bigram corpus perplexity 304.1673339728869


# Exercise 3. Bigram model with general additive (Lidstone) add-k smoothing

Modify your code from Exercise 2 such that it generalizes beyond adding 1 to all counts, but can add differing amounts k of mass instead, to implement general additive add-k smoothing.

On the HELDOUT corpus `switchboard_language_model_heldout.txt` experiment with different values of k (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment.

Once you find the value which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the test data once and report the scores in a comment.

In [25]:
sentences = switchboard_lm_train_oov
# First get the counts from the training corpus for bigrams without smoothing
bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
delta = 1  # delta is order - 1
for s in sentences:
    words = tokenize_sentence(s, 2)  # tokenize sentence with the order 2 as the parameter
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        bigrams[glue_tokens(ngram, 2)] +=1
        bigram_context[glue_tokens(context, 1)] += 1
print(len(bigrams.keys()), "different bigrams")
print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

211560 different bigrams
12730 different bigram contexts (and unigrams) observed


In [26]:
# a handy function to calculate the probability of an bigram from the counts
def prob_bigram_MLE(ngram,k):
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)] + k
    denominator = bigram_context[glue_tokens(ngram[:1], 1)] + (k*len(vocab_words))
    prob = numerator / denominator
    return prob


# # check if each bigram continuation distribution sums to one
# # look at the distributions of possible continuations after each word
# for context, v in bigram_context.items():
#     context = unglue_tokens(context, 1)
#     #print("%% context", context)
#     check_ngram_total_sums_to_1 = 0
#     # for a given context the continuation probabilities 
#     # over the whole vocab should sum to 1
#     for u in unigrams.keys():
#         ngram = context + [u]
#         prob = prob_bigram_MLE(ngram)
#         #print(ngram, prob)
#         check_ngram_total_sums_to_1 += prob
#     #print("sums to 1?", check_ngram_total_sums_to_1)

In [27]:
sentences = switchboard_lm_heldout_oov
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s
for k in [0.2, 0.4, 0.6, 0.8]:
  for sent in sentences:
      words = tokenize_sentence(sent, 2)
      #print(words)
      sent_s = 0  # recording non-normalized entropy for this sentence
      sent_N = 0  # total number of words in this sentence (for normalization)
      for i in range(delta, len(words)):
          context = words[i-delta:i]
          target = words[i]
          ngram = context + [target]
          #print(ngram)
          prob = prob_bigram_MLE(ngram,k)
          #print(prob)
          s += -log(prob, 2) # add the neg log prob to s
          sent_s += -log(prob, 2)  # add the neg log prob to sent_s
          N += 1 # increment the number of total words
          sent_N += 1 # increment the number of total words in this sentence
      sent_cross_entropy = sent_s/sent_N
      sent_perplexity = 2 ** sent_cross_entropy
      #print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
  cross_entropy = s/N
  perplexity = 2 ** cross_entropy
  print("for k:  ",k)
  print("bigram corpus cross entropy", cross_entropy)
  print("bigram corpus perplexity", perplexity)
  print("*"*8)

for k:   0.2
bigram corpus cross entropy 7.439684502618534
bigram corpus perplexity 173.60738423639475
********
for k:   0.4
bigram corpus cross entropy 7.596623829470926
bigram corpus perplexity 193.55822840126572
********
for k:   0.6
bigram corpus cross entropy 7.721658611115744
bigram corpus perplexity 211.08183366073382
********
for k:   0.8
bigram corpus cross entropy 7.826616103636395
bigram corpus perplexity 227.01063930330494
********



# Exercise 4. Ngram models with Kneser-Ney smoothing

Kneser-Ney smoothing is a state-of-the-art technique for smoothing n-gram models.

The algorithm is quite complicated, and is implemented for you below for training/getting the appropriate counts  on the training data (`ngrams_interpolated_kneser_ney()`). The training process is implemented below for a trigram model, but without doing the appropriate out-of-vocab word replacement as you've done above.

The application at test time is done with the method `kneser_ney_ngram_prob()` using the trained Counters, which gives the probability of the model applied to an ngram of a given order, with a given discount.

Try to follow how the training works and how the application of the model to ngrams works, and refer to the below article on QM plus (pages 7-8 particularly):

"A Bit of Progress in Language Modeling" (2001) - Joshua T. Goodman

In this exercise, you will first modify the training part of the code so it does the replacement of out-of-vocab words as you did in the previous exercises. You do not need to modify the methods below.

On the HELDOUT corpus experiment with different orders from trigram upwards (try 3, 4 and 5) and different discount values (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment. 

Once you find the order and discount values which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the TEST data once and report the scores in a comment.

In [40]:
# Kneser-Ney smoothing
order = 3
discount = 0.8

unigram_denominator = 0
ngram_numerator_map = Counter() 
ngram_denominator_map = Counter() 
ngram_non_zero_map = Counter()


def ngrams_interpolated_kneser_ney(tokens,
                                   order,
                                   ngram_numerator_map,
                                   ngram_denominator_map,
                                   ngram_non_zero_map,
                                   unigram_denominator):
    """Function used in n-gram language model training
    to count the n-grams in tokens and also record the
    lower order non -ero counts necessary for interpolated Kneser-Ney
    smoothing.
    
    Taken from Goodman 2001 and generalized to arbitrary orders"""
    for i in range(order-1,len(tokens)): # tokens should have a prefix of order - 1
        print(i)
        for d in range(order,0,-1): #go through all the different 'n's
            if d == 1:
                unigram_denominator += 1
                ngram_numerator_map[glue_tokens(tokens[i],d)] += 1
            else:
                den_key = glue_tokens(tokens[i-(d-1) : i], d)
                num_key = glue_tokens(tokens[i-(d-1) : i+1], d)
    
                ngram_denominator_map[den_key] += 1
                # we store this value to check if it's 0
                tmp = ngram_numerator_map[num_key]
                ngram_numerator_map[num_key] += 1 # we increment it
                if tmp == 0: # if this is the first time we see this ngram
                    #number of types it's been used as a context for
                    ngram_non_zero_map[den_key] += 1
                else:
                    break 
                    # if the ngram has already been seen
                    # we don't go down to lower order models
    return ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator


In [41]:
# train the model
# corpus = open("switchboard_lm_train.txt")
corpus = switchboard_lm_train_oov
for line in corpus:
    tokens = tokenize_sentence(line, order)
    ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator =\
            ngrams_interpolated_kneser_ney(tokens,
                                           order,
                                           ngram_numerator_map,
                                           ngram_denominator_map,
                                           ngram_non_zero_map,
                                           unigram_denominator)
# corpus.close() 

2
3
4
5
6
2
3
4
5
6
7
8
9
2
3
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
5
2
3
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
3
4
2
3
4
5
6
7
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2
3
4
5
6
2
3
4
5
6
7
8
2
3
4
5
6
7
8
2
3
2
3
4
5
6
7
2
3
4
5
6
7
8
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
2
3
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6
7
8
9
10
2
3
4
5
2
3
4
5
6
7
8
9
2
3
2
3
4
2
3
4
5
2
3
4
5
6
7
8
9
2
3
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2
3
4
5
6
7
8
9
10
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
2
3
4
5
2
3
4
5
6
7
8
9
10
11
2
3
4
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
5
6
7
8
9
10
2
3
2
3
4
5
2
3
4
5
6
7
2
3
4
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2
3
4
5
6
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
1

4
5
6
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
2
3
4
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
2
3
2
3
4
5
6
7
8
9
10
2
3
4
5
6
7
8
9
10
2
3
2
3
4
5
6
7
8
2
3
4
5
6
7
2
3
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
2
3
4
2
3
4
5
6
7
8
2
3
4
5
6
7
8
2
3
2
3
4
5
6
2
3
4
5
6
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
2
3
2
3
2
3
2
3
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2
3
2
3
4
5
6
7
2
3
4
5
6
2
3
2
3
4
2
3
2
3
4
5
6
7
8
9
10
11
12
2
3
4
2
3
4
5
6
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
2
3
4
5
6
2
3
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2
3
4
5
2
3
4
5
6
7
8
2
3
4
5
6
7
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
2
3
2
3
2
3
4
5
6
7
8
9
10
2
3
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
2
3
2
3
2
3
4
5
6
7
8
9


4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
4
5
6
7
8
9
10
11
2
3
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
2
3
4
5
6
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
2
3
2
3
2
3
2
3
2
3
4
5
6
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
2
3
2
3
2
3
4
2
3
2
3
4
5
6
7
8
9
10
11
12
2
3
2
3
2
3
2
3
2
3
4
5
6
7
2
3
4
5
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
2
3
2
3
2
3
4
2
3
4
5
6
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
2
3
4
5
6
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
2
3
2
3
4
5
2
3
4
5
6
7
8
9
10
2
3
4
5
6
2
3
4
5
6
2
3
2
3
2
3
4
5
6
7
8
9
10
2
3
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
2
3
4
5
2
3
4
5
2
3
4
5
6
7
8
9
10
11
12
2
3
2
3
2
3
4
5
6
2
3
4
5
2
3
4
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
2
3
2
3
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
2
3
4
5
2
3
2
3
4
5
6
7
8
9


5
6
2
3
4
5
6
7
2
3
4
5
6
7
2
3
4
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
7
8
9
2
3
4
5
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6
2
3
4
2
3
4
5
6
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
2
3
4
5
2
3
4
2
3
2
3
4
2
3
2
3
4
5
2
3
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
2
3
2
3
4
5
6
2
3
4
5
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
2
3
4
5
6
7
8
9
10
2
3
4
5
6
7
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
2
3
4
5
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
2
3
4
2
3
2
3
2
3
4
5
2
3
2
3
4
2
3
4
5
6
7
8
2
3
4
2
3
2
3
4
5
6
7
8
9
10
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
2
3
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6

3
4
5
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
5
6
7
8
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
2
3
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
2
3
4
5
6
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
2
3
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
2
3
2
3
2
3
4
5
6
7
8
9
10
11
1

...

5
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
2
3
4
5
6
7
8
9
2
3
2
3
4
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2
3
4
5
6
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
2
3
4
5
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
7
2
3
4
5
6
7
8
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
2
3
4
5
6
7
8
2
3
2
3
2
3
4
5
6
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
4
5
6
2
3
4
5
6
7
2
3
4
5
2
3
2
3
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
2
3
4
5
6
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
2
3
2
3
4
5
6
2


2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2
3
2
3
2
3
4
5
6
7
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
2
3
2
3
4
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
2
3
4
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
2
3
4
5
6
7
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
9
10
2
3
4
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
2
3
2
3
2
3
2
3
4
5
6
7
8
2
3
4
5
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6
7
8
9
10
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
8
9
10
11
12
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2
3
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
2
3
2
3
4
5
6
2
3
4
5
6
7
8
9
10
11
12
13
2
3
2
3
2
3
4
5
6
7
8
9
2
3
4
2
3
2
3
4
2
3
4
5
6
7
2
3
2
3
4
5
6
7
8
2
3
2
3
4
5
6
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1

4
5
6
7
8
2
3
2
3
2
3
2
3
4
5
6
7
2
3
2
3
4
5
6
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
5
6
2
3
4
5
6
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
2
3
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2
3
2
3
4
5
2
3
4
5
6
7
8
2
3
2
3
4
2
3
4
5
6
2
3
4
5
2
3
4
5
6
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
7
8
9
10
11
12
13
2
3
4
5
6
2
3
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
3
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
2
3
4
2
3
4
5
6
7
8
9
2
3
2
3
4
5
2
3
2
3
4
5
6
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
7
8
9
10
11
2
3
2
3
4
5
2
3
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2
3
2
3
4
5
6
7
8
9
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
4
5
6
7
8
9
2
3
2
3
4
5
2
3
4
5
6
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
2
3
4
5
6
2
3
4
5
2
3
4
5
6
7
2
3
4
5
2
3
4
5
6
7
8
2
3
2

2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
2
3
4
2
3
4
5
6
7
8
9
10
11
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
2
3
4
5
2
3
4
5
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2
3
4
5
6
7
8
9
2
3
2
3
4
2
3
4
5
6
7
2
3
4
5
6
7
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
2
3
2
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
2
3
4
5
6
7
8
9
10
11
12
13
14
2
3
2
3
4
5
6
7
8
2
3
4
2
3
4
5
6
7
8
2
3
2
3
4
5
2
3
4
5
6
7
8
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2
3
4
5
2
3
4
5
6
7
8
9
10
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
2
3
4
5
6
7
8
9
10
2
3
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
2
3
2
3
4
5
6
7
8
9
2
3
4
5
2
3
4
5
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
2
3
2
3


KeyboardInterrupt: 

In [30]:
def kneser_ney_ngram_prob(ngram, discount, order):
    """KN smoothed ngram probability from Goodman 2001.
    This is run at test time to calculate the probability
    of a given n-gram or a given order with a given discount.
    
    ngram :: list of strings, the ngram
    discount :: float, the discount used (lambda)
    order :: int, order of the model
    """
    # First, calculate the unigram prob of the last token 
    # If we've never seen it at all, it will 
    # have no probability as a numerator
    uni_num = ngram_numerator_map.get(glue_tokens(ngram[-1], 1))
    if not uni_num: # if no value found in dict, make it 0
        uni_num = 0
    probability = previous_prob = float(uni_num) / float(unigram_denominator)
    
    # Given <unk/> should have been used in place of unknown words before passing
    # to this method,
    # probability should be non-zero
    if probability == 0.0:
        print("0 prob for unigram!")
        print(glue_tokens(ngram[-1], 1))
        print(ngram)
        print(ngram_numerator_map.get(glue_tokens(ngram[-1], 1)))
        print(unigram_denominator)
        raise Exception

    # Compute the higher order probs (from 2/bi-gram upwards) and interpolate them
    for d in range(2,order+1):
        # Get the number of times this denominator has been seen as one
        # For bigrams this is the number of different continuation types counted
        ngram_den = ngram_denominator_map.get(glue_tokens(ngram[-(d):-1], d))
        if not ngram_den: # if no value found in dict, make it 0
            ngram_den = 0
        if ngram_den != 0: 
            ngram_num = ngram_numerator_map.get(glue_tokens(ngram[-(d):], d))
            if not ngram_num: # if no value found in dict, make it 0
                ngram_num = 0
            if ngram_num != 0:
                current_prob = (ngram_num - discount) / float(ngram_den)
            else:
                current_prob = 0.0
            nonzero = ngram_non_zero_map.get(glue_tokens(ngram[-(d):-1], d))
            if not nonzero: # if no value found in dict, make it 0
                nonzero = 0
            # interpolate with previous probability of lower orders calculated
            # so far
            current_prob += nonzero * discount / ngram_den * previous_prob
            previous_prob = current_prob
            probability = current_prob
        else:
            #if this context (e.g. bigram contect for trigrams) has never been seen, 
            #then we can only get the last order with a probability (e.g. unigram)
            #and halt
            probability = previous_prob
            break
    return probability

In [36]:
#ngram_numerator_map,
#ngram_denominator_map,
#ngram_non_zero_map,
#unigram_denominator
# In this exercise, you will first modify the training part of 
# the code so it does the replacement of out-of-vocab words as 
# you did in the previous exercises. You do not need to modify the methods below.

# On the HELDOUT corpus experiment with different orders from
# trigram upwards (try 3, 4 and 5) and different discount values
# (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and
# report the perplexity scores for these different values in a comment.

# Once you find the order and discount values which gives you 
# the lowest perplexity on the heldout data, use this model to
# get the perplexity of the TEST data once and report the scores in a comment.
delta = 2
discount = 0.2
order = 3
sentences = switchboard_lm_heldout_oov
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s

for sent in sentences:
    print(sent)
    words = tokenize_sentence(sent, order)
    print(words)
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for i in range(delta, len(words)):
        #print(words)
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        print(ngram)
        break
        prob = kneser_ney_ngram_prob(ngram, discount, order)
        #print(prob)
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    #print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("bigram corpus cross entropy", cross_entropy)
print("bigram corpus perplexity", perplexity)

do you know anyone that is in a nursing home or has ever been in $unc$one
['<s>', '<s>', 'do', 'you', 'know', 'anyone', 'that', 'is', 'in', 'a', 'nursing', 'home', 'or', 'has', 'ever', 'been', 'in', '$unc$one', '</s>']
['<s>', '<s>', 'do']


ZeroDivisionError: division by zero